# 文字轉語音

In [1]:
import requests
import json

In [16]:
#將上面的金鑰貼入單引號中
subscription_key = '8535969260fa41aa9b11b67b73c7790a'

text2speech_api_url = 'https://japaneast.api.cognitive.microsoft.com/sts/v1.0/issuetoken'

## 取得網站的授權

底下的程式碼會取得網站的授權。  
由於微軟的語音服務功能分散在不同的網站，為了讓使用者可以存取這些服務，  
我們要先跟 Azure 取得一個「授權碼」。

之後再用這個授權碼連到微軟的語音服務網站。  
微軟的網址會檢查這個授權碼，通過後才能使用它的服務。

這個授權碼每隔十分鐘會失效。  
所以要是呼叫發生失敗時，請再重新取得一次授權碼。 

In [17]:
headers = {
        'Ocp-Apim-Subscription-Key': subscription_key
    }
response = requests.post(text2speech_api_url, headers=headers)
access_token = str(response.text)
print(access_token)

eyJhbGciOiJodHRwOi8vd3d3LnczLm9yZy8yMDAxLzA0L3htbGRzaWctbW9yZSNobWFjLXNoYTI1NiIsInR5cCI6IkpXVCJ9.eyJyZWdpb24iOiJqYXBhbmVhc3QiLCJzdWJzY3JpcHRpb24taWQiOiI1ZjYxNTkzNGVmNmM0MjhmOGY5NzEwN2EzYjMyNGZjZSIsInByb2R1Y3QtaWQiOiJTcGVlY2hTZXJ2aWNlcy5GMCIsImNvZ25pdGl2ZS1zZXJ2aWNlcy1lbmRwb2ludCI6Imh0dHBzOi8vYXBpLmNvZ25pdGl2ZS5taWNyb3NvZnQuY29tL2ludGVybmFsL3YxLjAvIiwiYXp1cmUtcmVzb3VyY2UtaWQiOiIvc3Vic2NyaXB0aW9ucy9iMTg1NmJmNy01ZTY1LTQxNzItODQ2Yy05NTJmYWEwNmEzNmUvcmVzb3VyY2VHcm91cHMvVGVhY2hlckdycC9wcm92aWRlcnMvTWljcm9zb2Z0LkNvZ25pdGl2ZVNlcnZpY2VzL2FjY291bnRzL0lFTVQyUyIsInNjb3BlIjoic3BlZWNoc2VydmljZXMiLCJhdWQiOiJ1cm46bXMuc3BlZWNoc2VydmljZXMuamFwYW5lYXN0IiwiZXhwIjoxNTkwNTAyNjgxLCJpc3MiOiJ1cm46bXMuY29nbml0aXZlc2VydmljZXMifQ.pZJ15KeNnum5QTedpqw-16Qqx4OoyPGU5fU04Rv82Dk


上面的程式碼就展示了授權碼。 

## 執行文字轉語音的功能

底下的程式碼的部分細節，請參考[微軟API的說明](https://docs.microsoft.com/zh-tw/azure/cognitive-services/speech-service/rest-text-to-speech#audio-outputs)。

底下的 ``ttsurl`` 是微軟合成語音的網址，我們在這個專案中，選擇連到東日本的伺服器來完成語音合成的功能。

在[這裏](https://docs.microsoft.com/zh-tw/azure/cognitive-services/speech-service/rest-text-to-speech#audio-outputs)列出了提供語音合成伺服器的網址。  
有些伺服器還提供「類人聲」的語音合成服務。不過目前東日本的伺服器只有較生硬的語音合成服務 (聽起來類似 Google 小姐的聲音)。

程式碼 header 部分，有 access_token。這個就是在上面程式碼中取得的授權碼 (見下圖)。  
我們需要這個授權碼，才能存取微軟語音合成的服務。

![](https://i.imgur.com/m95Ba5A.png)

Header 中的 User-Agent 是由瀏覽器來決定。要知道自己機器上的 User-Agent 為何，  
請在瀏覽器中按下右鍵，並點「檢查」(以 Chrome 為例)

![](https://i.imgur.com/L7kCoAt.png)

接著請依照下圖的指示，就可以取得自己系統上的 User-Agent 了。

![](https://i.imgur.com/LNtT2kd.png)

接著我們就要把文字送到網站去，請網站幫我們轉成語音檔。  
依照微軟的說明，轉檔後的音訊不能超過 10 分鐘，超過 10 分鐘的語音會被截掉。  

微軟要求傳送的文字要以``語音合成標記語言 (SSML)`` 來傳送，其細節請見[這裏](https://docs.microsoft.com/zh-tw/azure/cognitive-services/speech-service/speech-synthesis-markup?tabs=csharp)，在這份文件中就不再多加說明。

SSML 基本上是一個 XML 的文件。  
本專案要傳的文件如下所示。 

```xml
<speak version="1.0" xml:lang="zh-TW" xmlns="http://www.w3.org/2001/10/synthesis">  
  <voice name="zh-TW-HanHanRUS">
    您好嗎？我很好
  </voice>  
</speak>  
```

由於是 XML 文件，我們可以使用 Python 內建的套件 ElementTree 來幫我們建立這個 XML。  
底下的程式碼會建立一個 xml 最上層的標籤，稱為 speak。

```Python
xml_body = ElementTree.Element('speak', version='1.0')
```
所建立的標籤如下所示：

```xml
<speak version="1.0">
</speak>
```

接著

```Python
xml_body.set('xmlns', 'http://www.w3.org/2001/10/synthesis')
xml_body.set('xml:lang', 'zh-TW')
```
會把標籤中的屬性補齊，所以整個 XML 文件如下所示：  

```xml
<speak version="1.0" xml:lang="zh-TW" xmlns="http://www.w3.org/2001/10/synthesis">
</speak>
```
程式碼會在 speak 標籤下，建立一個子標籤 (SubElement)。並且填入對應的屬性。  

```Python
xml_voice = ElementTree.SubElement(xml_body, 'voice')
xml_voice.set('name','zh-TW-HanHanRUS')
```

所得到的 XML 文件如下所示：

```xml
<speak version="1.0" xml:lang="zh-TW" xmlns="http://www.w3.org/2001/10/synthesis">
<voice name="zh-TW-HanHanRUS">
    
</voice>  
</speak>
```

XML 文件中的 ``zh-TW-HanHanRUS`` 代表語音是以中文女性的合成語音唸出來。  
相關的代號請見[這裏](https://docs.microsoft.com/zh-tw/azure/cognitive-services/speech-service/language-support#text-to-speech)的說明。  

最後，xml_voice.text 則可以指定你要轉成語音的文字。  
指定後 XML 文件如下所示：  

```xml
<speak version="1.0" xml:lang="zh-TW" xmlns="http://www.w3.org/2001/10/synthesis">  
  <voice name="zh-TW-HanHanRUS">
    您好嗎？我很好
  </voice>  
</speak>  
```

接著，我們使用 tostring 把 XML　文件轉成文字，然後用 unicode 的方式來編碼。  
在網路上傳送時，由於裏面包含了中文字，所以要使用 utf-8 來再編碼一次。

In [23]:
from xml.etree import ElementTree
from xml.etree.ElementTree import tostring
ttsurl='https://japaneast.tts.speech.microsoft.com/cognitiveservices/v1'

headers = {
            'Authorization': 'Bearer ' + access_token,
            'Content-Type': 'application/ssml+xml',
            'X-Microsoft-OutputFormat': 'riff-24khz-16bit-mono-pcm',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
        }
xml_body = ElementTree.Element('speak', version='1.0')

xml_body.set('xmlns', 'http://www.w3.org/2001/10/synthesis')
xml_body.set('xml:lang', 'zh-TW')
xml_voice = ElementTree.SubElement(xml_body, 'voice')
xml_voice.set('name','zh-TW-HanHanRUS')
xml_voice.text = '您好嗎？我很好'

print(tostring(xml_body, encoding="unicode", method="html"))
body = tostring(xml_body, encoding="unicode", method="html")
body = body.encode('utf-8')

<speak version="1.0" xml:lang="zh-TW" xmlns="http://www.w3.org/2001/10/synthesis"><voice name="zh-TW-HanHanRUS">您好嗎？我很好</voice></speak>


將前面所組出來的資料，送到 ttsurl 所指定的雲端。 

之後微軟的伺服器會將文字轉成語音，並回傳。  

In [24]:
response = requests.post(ttsurl, headers=headers, data=body)

## 儲存語音

我們開啟一個檔案，將微軟回傳的語音寫入成 wav 檔。

In [25]:
with open('sample-001.wav', 'wb') as audio:
    audio.write(response.content)